In [1]:
from pyspark.sql import SparkSession
from operator import add
import pyspark.sql.functions as f
from functools import reduce

In [2]:
# New API
spark_session = SparkSession\
        .builder\
        .appName("G11_testingAnalysis")\
        .config("spark.executor.cores",2)\
        .config("spark.cores.max", 4)\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled", True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-03-07 14:57:06,161 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
2022-03-07 14:57:25,565 WARN spark.ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.
2022-03-07 14:57:25,632 WARN cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM 

In [3]:
data_frame = spark_session.read.csv('hdfs://192.168.2.112:9000/user/ubuntu/million_song_dataset/million_song_subset_1gb_1.csv')
data_frame.show()

2022-03-07 14:58:11,787 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------------+-------------------+-------------------+------------------+--------------------+---------------+-----------------+---------------+----------------+----------------+--------------------+--------------------+------------------+-------------------+--------------------+----------------+---------------+--------------------+-----------------+-------------------+--------------------+--------------------+------------+--------+--------------+------+----+--------------+--------+----+---------------+-----------------+-------+--------------+--------------------+------------------+--------------+-------------------+----------------+---------------+--------------------+--------------------+--------------------+--------------+-------------------+-----------+----------------+----------+---------------+------------+-----------------+-------------+-------------------+----+
|               _c0|                _c1|                _c2|               _c3|                 _c4|            

In [106]:
# Mostly python stuff. Need to convert it to pyspark.
import pyspark.sql.functions as f
from functools import reduce

# Convert the genre string into a list.
def get_genre_list(genre_row):
    #genre_str = genre_row[1].asDict()['_c17']
    genre_list = genre_row.split('" "')
    genre_list = [genre.strip(' ').strip('"') for genre in genre_list]
    return genre_list

# Return avg value from a list of values.
def average(lst):
    return reduce(lambda a, b: a + b, lst) / len(lst)


# _c2 = hotness
# _c17 = genres
# _c10 = artist name
# _c14 = song name

# Retrieve the data.
data = data_frame.select('_c10', '_c14', '_c2', '_c17').take(100)
hotness_list = [float(x[2]) for x in data[1:]]
artist_name_list = [x[0] for x in data[1:]]
song_title_list = [x[1] for x in data[1:]]
artist_genres_list = [x[3] for x in data[1:]]

# Put the genre in a dict as key and the value is a list of all of the hotnesses of that genre.
genre_hotness_dict = {}
for iteration in range(len(hotness_list)):
    genre_list = get_genre_list(artist_genres_list[iteration])
    
    for genre in genre_list:
        if genre in genre_hotness_dict:
            genre_hotness_dict[genre].append(hotness_list[iteration])
        else:
            genre_hotness_dict[genre] = [hotness_list[iteration]]

# Put it all in a new dict that will only have the genre name and avg hotness.
genre_hotness_avg_dict = {}
for genre in genre_hotness_dict:
    hotness_list = genre_hotness_dict[genre]
    
    # Ignore genres that have too small of a sample size of hotness values.
    if not len(hotness_list) < 2:
        genre_hotness_avg_dict[genre] = average(hotness_list)

# sort the dictionary, highest to lowest.
sorted_genre_hotness_avg_dict = {k: v for k, v in sorted(genre_hotness_avg_dict.items(), key=lambda item: item[1], reverse=True)}
print(sorted_genre_hotness_avg_dict)



{'party music': 0.6494402190494821, 'quiet storm': 0.6450086147891794, 'album rock': 0.6066644455652956, 'arena rock': 0.5861452694802278, 'experimental rock': 0.5826569613074153, 'grunge': 0.5812051854038595, 'song writer': 0.5774666907682452, 'alternative metal': 0.572106561831481, 'dark': 0.571586627872874, 'old': 0.5660538269915196, 'soft': 0.5657255496005742, 'urban': 0.5641364018896896, 'heavy': 0.5623624144321281, 'piano': 0.5615372276395678, 'melodic hardcore': 0.5567834536778039, 'aggressive': 0.5464057952100154, 'adult contemporary': 0.5429356574855367, 'solo': 0.5422754056737691, 'melodic': 0.5415558374205299, '00s': 0.541456259908682, 'neo soul': 0.5395717150608524, 'beautiful': 0.5394339120413254, 'trumpet': 0.537318992604052, 'melancholia': 0.5359536766582431, 'theater': 0.5341928878451317, 'crossover': 0.5334071991355699, 'acoustic': 0.5329628236034331, 'power pop': 0.5324342267868635, 'noise': 0.5304930912336521, 'epic': 0.5302153891472451, 'emotional': 0.52923769690121

In [5]:
# Intermittent solution. Using more pyspark but also some python.

# Return avg value from a list of values.
def average(lst):
    return reduce(lambda a, b: a + b, lst) / len(lst)


def trim_genre(genre):
    return genre.strip(' ').strip('"')

trim_genre = f.udf(trim_genre)


# _c2 = hotness
# _c17 = genres
# _c10 = artist name
# _c14 = song name


hotness_and_genres_df = data_frame.select(f.split(trim_genre(data_frame['_c17']), '" "'), '_c2')\
                        .filter(data_frame['_c17'] != 'artist_terms')\
                        .withColumn('_c2', data_frame['_c2'].cast('float').alias('_c2'))\
                        .rdd.map(lambda x: x).take(1000)


# Put the genre in a dict as key and the value is a list of all of the hotnesses of that genre.
genre_hotness_dict = {}
#for iteration in range(len(hotness_df)):
for x in hotness_and_genres_df:
    #genre_list = artist_genres_df[iteration][0]
    genre_list = x[0]
    hotness = x[1]
    for genre in genre_list:
        if genre in genre_hotness_dict:
            genre_hotness_dict[genre].append(hotness)
        else:
            genre_hotness_dict[genre] = [hotness]

# Put it all in a new dict that will only have the genre name and avg hotness.
genre_hotness_avg_dict = {}
for genre in genre_hotness_dict:
    hotness_list = genre_hotness_dict[genre]
    
    # Ignore genres that have too small of a sample size of hotness values.
    if not len(hotness_list) < 2:
        genre_hotness_avg_dict[genre] = average(hotness_list)

# sort the dictionary, highest to lowest.
sorted_genre_hotness_avg_dict = {k: v for k, v in sorted(genre_hotness_avg_dict.items(), key=lambda item: item[1], reverse=True)}
print(sorted_genre_hotness_avg_dict)

{'rap metal': 0.6726094782352448, 'warm': 0.6136556267738342, 'moody': 0.6051762342453003, 'fresh': 0.6048798362414042, 'posi': 0.5656673709551493, 'emotional': 0.5644906063874563, 'clever': 0.5602283875147501, 'intense': 0.5550181865692139, 'dance pop': 0.5508506265468895, 'east coast rap': 0.5482332223937625, 'boy band': 0.5460673073927561, 'heavy': 0.5458914550867948, 'rapcore': 0.5428807139396667, 'american trad rock': 0.5404893926211766, 'teen pop': 0.5397439310948055, '80s pop': 0.5394190102815628, 'midwest rap': 0.5376891096433004, 'producer': 0.5375431138893654, 'beats': 0.5365706028476838, 'black': 0.5333028456994465, 'christian hardcore': 0.5329811771710714, 'loud': 0.5309211286631498, 'golden age': 0.5302402079105377, 'smooth': 0.5293156516795255, 'nashville': 0.5273650884628296, 'urban': 0.5264819672992153, 'aggressive': 0.5235328115522861, 'wave': 0.5228735208511353, 'influential': 0.5225525230169297, 'new romantic': 0.5213451782862345, 'sentimental': 0.5193828009068966, '

In [5]:
# Solution that does almost all of the calculations using pyspark, it tends to crash though and does not aggregate.
def trim_genre(genre_list):
    return [genre.strip(' ').strip('"') for genre in genre_list]

def trim_genre_2(genre):
    return genre.strip(' ').strip('"')

def mapper(line):
    return ((word, line[1]) for word in line[0])

trim_genre = f.udf(trim_genre_2)

genres_and_hotness_df = data_frame.select(f.split(trim_genre(data_frame['_c17']), '" "'), '_c2')\
                        .filter(data_frame['_c17'] != 'artist_terms')\
                        .withColumn('_c2', data_frame['_c2'].cast('float').alias('_c2'))\
                        .rdd.map(lambda x: x)

mappings = genres_and_hotness_df.flatMap(mapper)
print(mappings.take(200))
#mappings_reduced = mappings.reduceByKey(add)
#print(mappings_reduced.take(1))

[('chanson', 0.38615164160728455), ('visual kei', 0.38615164160728455), ('hip hop', 0.38615164160728455), ('pop rock', 0.38615164160728455), ('british pop', 0.38615164160728455), ('jrock', 0.38615164160728455), ('world music', 0.38615164160728455), ('downtempo', 0.38615164160728455), ('singer-songwriter', 0.38615164160728455), ('latin pop', 0.38615164160728455), ('french', 0.38615164160728455), ('male vocalist', 0.38615164160728455), ('alternative rock', 0.38615164160728455), ('chill-out', 0.38615164160728455), ('rock', 0.38615164160728455), ('french pop', 0.38615164160728455), ('ethnic fusion', 0.38615164160728455), ('electronica', 0.38615164160728455), ('pop', 0.38615164160728455), ('acoustic', 0.38615164160728455), ('chanson francaise', 0.38615164160728455), ('japanese', 0.38615164160728455), ('j pop', 0.38615164160728455), ('alternative', 0.38615164160728455), ('indie', 0.38615164160728455), ('instrumental', 0.38615164160728455), ('spanish', 0.38615164160728455), ('french rock', 0.